In [1]:
import os
import json
import subprocess
import numpy as np
import pandas as pd
import pickle as pkl
import nibabel as nib
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from os.path import join as pjoin
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from nilearn.glm.first_level import make_first_level_design_matrix

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams.update({'font.size': 12, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

def save_ciftifile(data, filename):
    template = '/nfs/z1/zhenlab/BrainImageNet/NaturalObject/data/bold/Analysis_derivatives/ciftify/sub-core02/MNINonLinear/Results/ses-ImageNet01_task-object_run-1/ses-ImageNet01_task-object_run-1_Atlas.dtseries.nii'
    ex_cii = nib.load(template)
    ex_cii.header.get_index_map(0).number_of_series_points = data.shape[0]
    nib.save(nib.Cifti2Image(data.astype(np.float32), ex_cii.header), filename)

# define path
beta_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/beta'
fmriprep_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/fmriprep'
ciftify_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/ciftify'
nifti_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/nifti'
result_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/result'

/usr/local/neurosoft/anaconda3/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [3]:
from scipy.stats import pearsonr

# Load beta for 30 subjects 
sub_names = sorted([i for i in os.listdir(beta_path) if i.startswith('sub')])
mask_sum = ['EVC', 'VTC', 'LO', 'AIP']
n_sub = len(sub_names)
n_class = 180

beta_sum = np.zeros((n_sub, n_class, 59412))
for sub_idx, sub_name in enumerate(sub_names):
    # define beta path
    beta_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_action-beta_clean.npy')
    beta_sub = np.load(beta_sub_path)
    scaler = StandardScaler()
    beta_sum[sub_idx] = scaler.fit_transform(beta_sub)
    print(f'Finish {sub_name}')

beta_sum = beta_sum.mean(axis=0)

Finish sub-01
Finish sub-02
Finish sub-03
Finish sub-04
Finish sub-05
Finish sub-06
Finish sub-07
Finish sub-08
Finish sub-09
Finish sub-10
Finish sub-11
Finish sub-12
Finish sub-13
Finish sub-14
Finish sub-15
Finish sub-16
Finish sub-17
Finish sub-18
Finish sub-19
Finish sub-20
Finish sub-21
Finish sub-22
Finish sub-23
Finish sub-24
Finish sub-25
Finish sub-26
Finish sub-27
Finish sub-28
Finish sub-29
Finish sub-30


In [2]:
# load animacy labels
beh_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/utils/action_spaces/behavior'
beh_df = pd.read_csv(pjoin(beh_path, 'dim_scores.csv'), index_col=0)
beh_df['mean_score'] = beh_df.iloc[:, 2:].mean(axis=1)
beh_df = beh_df.drop(beh_df.columns[2:22], axis=1)
beh_df = beh_df.sort_values(by='class')

#### Generate brain map using contrast substraction

In [ ]:
dims = ['transitivity', 'sociality']
brain_map = np.zeros((2, beta_sum.shape[1]))
for dim_idx, dim in enumerate(dims):
    # get label
    dim_score = beh_df.loc[beh_df['dim']==dim, 'mean_score'].to_numpy()
    dim_content = np.array([1 if item > 4 else 0 for item in dim_score])
    # # compute brain map
    brain_map = np.zeros((1, 91282))
    brain_map[0, :59412] = beta_sum[dim_content==1].mean(axis=0) - beta_sum[dim_content==0].mean(axis=0)
    save_ciftifile(brain_map, pjoin(result_path, f'{dim}_map.dtseries.nii'))
    print('%s map threshold: left: %.2f; right: %.2f'%(dim, brain_map.squeeze()[:59412].mean() - brain_map.squeeze()[:59412].std(),
                                                       brain_map.squeeze()[:59412].mean() + brain_map.squeeze()[:59412].std()))


#### Generate brain map using correlation

In [6]:
from scipy.stats import pearsonr
import subprocess

dims = ['transitivity', 'sociality']
brain_map = np.zeros((2, beta_sum.shape[1]))
p_value = 0.05
smooth_kernel = 8

for dim_idx, dim in enumerate(dims):
    # get label
    dim_score = beh_df.loc[beh_df['dim']==dim, 'mean_score'].to_numpy()
    # # compute brain map
    brain_map = np.zeros((1, 91282))
    for voxel_idx in range(59412):
        corr_info = pearsonr(beta_sum[:, voxel_idx], dim_score)
        if corr_info[1] < p_value:
            brain_map[0, voxel_idx] = corr_info[0]
    # smooth the target map
    out_file = pjoin(result_path, 'brain_map', f'{dim}_map_corr_smooth.dtseries.nii')
    save_ciftifile(brain_map, out_file)
    fsl_cmd = f'fslmaths {out_file} -s {smooth_kernel} -save'
    subprocess.check_call(fsl_cmd, shell=True)
